# Data Analysis of WeRateDogs Twitter archive

In [1]:
import pandas as pd
import requests
import os
import tweepy
import json
import time
import numpy as np

### Data Gathering 

In [2]:
#Download 'twitter-archive-enhanced.csv' manually and read to check if it was gathered successfuully.
twitter_arch = pd.read_csv('twitter-archive-enhanced.csv')
twitter_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
#Creates folder 'WeRateDogs' if it doesn't exist already.
#folder_name = 'WeRateDogs'
#if not os.path.exists(folder_name):
#    os.makedirs(folder_name)

In [4]:
#This creates
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(url.split('/')[-1]),mode='wb') as infile:
          infile.write(response.content)

In [4]:
image_df = pd.read_csv('image-predictions.tsv',sep='\t')
image_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [5]:
image_df.img_num.value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [6]:
twitter_arch.shape

(2356, 17)

In [7]:
image_df.shape

(2075, 12)

In [8]:
consumer_key = '********'
consumer_secret = '********'
access_token = '********'
access_secret = '********'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [41]:
tweet_json_col = 'id' + ' ' + 'retweet_count' + ' ' + 'favorite_count'
with open(os.path.join('tweet_json.csv'),mode='w') as infile:
    infile.write(tweet_json_col)
    infile.write('\n')

In [42]:
def gather(tw):
    tweet = api.get_status(twitter_arch.tweet_id[tw],tweet_mode='extended')
    tw_id = str(tweet._json['id'])
    retweet = str(tweet._json['retweet_count'])
    fav = str(tweet._json['favorite_count'])
    json_tweet = tw_id + " " + retweet + " " + fav
    print('{} of 2356 tweet ids queried'.format(tw+1))
    with open(os.path.join('tweet_json.csv'),mode='a') as outfile:
        outfile.write(json_tweet)
        outfile.write('\n') 
    
cnt = 0
missing = list()
start=time.time()

for tw in range(len(twitter_arch.tweet_id)):
    try:
        gather(tw)
    except:
        print('{} was not found'.format(tw))
        missing.append(twitter_arch.tweet_id[tw])
        
end=time.time()
print('Total time elapsed:{}'.format(start-end))

1 of 2356 tweet ids queried
2 of 2356 tweet ids queried
3 of 2356 tweet ids queried
4 of 2356 tweet ids queried
5 of 2356 tweet ids queried
6 of 2356 tweet ids queried
7 of 2356 tweet ids queried
8 of 2356 tweet ids queried
9 of 2356 tweet ids queried
10 of 2356 tweet ids queried
11 of 2356 tweet ids queried
12 of 2356 tweet ids queried
13 of 2356 tweet ids queried
14 of 2356 tweet ids queried
15 of 2356 tweet ids queried
16 of 2356 tweet ids queried
17 of 2356 tweet ids queried
18 of 2356 tweet ids queried
19 of 2356 tweet ids queried
19 was not found
21 of 2356 tweet ids queried
22 of 2356 tweet ids queried
23 of 2356 tweet ids queried
24 of 2356 tweet ids queried
25 of 2356 tweet ids queried
26 of 2356 tweet ids queried
27 of 2356 tweet ids queried
28 of 2356 tweet ids queried
29 of 2356 tweet ids queried
30 of 2356 tweet ids queried
31 of 2356 tweet ids queried
32 of 2356 tweet ids queried
33 of 2356 tweet ids queried
34 of 2356 tweet ids queried
35 of 2356 tweet ids queried
36 of 

282 of 2356 tweet ids queried
283 of 2356 tweet ids queried
284 of 2356 tweet ids queried
285 of 2356 tweet ids queried
286 of 2356 tweet ids queried
287 of 2356 tweet ids queried
288 of 2356 tweet ids queried
289 of 2356 tweet ids queried
290 of 2356 tweet ids queried
291 of 2356 tweet ids queried
292 of 2356 tweet ids queried
293 of 2356 tweet ids queried
294 of 2356 tweet ids queried
295 of 2356 tweet ids queried
296 of 2356 tweet ids queried
297 of 2356 tweet ids queried
298 of 2356 tweet ids queried
298 was not found
300 of 2356 tweet ids queried
301 of 2356 tweet ids queried
302 of 2356 tweet ids queried
303 of 2356 tweet ids queried
304 of 2356 tweet ids queried
305 of 2356 tweet ids queried
306 of 2356 tweet ids queried
307 of 2356 tweet ids queried
308 of 2356 tweet ids queried
309 of 2356 tweet ids queried
310 of 2356 tweet ids queried
311 of 2356 tweet ids queried
312 of 2356 tweet ids queried
313 of 2356 tweet ids queried
314 of 2356 tweet ids queried
315 of 2356 tweet ids 

557 of 2356 tweet ids queried
558 of 2356 tweet ids queried
559 of 2356 tweet ids queried
560 of 2356 tweet ids queried
561 of 2356 tweet ids queried
562 of 2356 tweet ids queried
563 of 2356 tweet ids queried
564 of 2356 tweet ids queried
565 of 2356 tweet ids queried
566 of 2356 tweet ids queried
566 was not found
568 of 2356 tweet ids queried
569 of 2356 tweet ids queried
570 of 2356 tweet ids queried
571 of 2356 tweet ids queried
572 of 2356 tweet ids queried
573 of 2356 tweet ids queried
574 of 2356 tweet ids queried
575 of 2356 tweet ids queried
576 of 2356 tweet ids queried
577 of 2356 tweet ids queried
578 of 2356 tweet ids queried
579 of 2356 tweet ids queried
580 of 2356 tweet ids queried
581 of 2356 tweet ids queried
582 of 2356 tweet ids queried
583 of 2356 tweet ids queried
584 of 2356 tweet ids queried
585 of 2356 tweet ids queried
586 of 2356 tweet ids queried
587 of 2356 tweet ids queried
588 of 2356 tweet ids queried
589 of 2356 tweet ids queried
590 of 2356 tweet ids 

832 of 2356 tweet ids queried
833 of 2356 tweet ids queried
834 of 2356 tweet ids queried
835 of 2356 tweet ids queried
836 of 2356 tweet ids queried
837 of 2356 tweet ids queried
838 of 2356 tweet ids queried
839 of 2356 tweet ids queried
840 of 2356 tweet ids queried
841 of 2356 tweet ids queried
842 of 2356 tweet ids queried
843 of 2356 tweet ids queried
844 of 2356 tweet ids queried
845 of 2356 tweet ids queried
846 of 2356 tweet ids queried
847 of 2356 tweet ids queried
848 of 2356 tweet ids queried
849 of 2356 tweet ids queried
850 of 2356 tweet ids queried
851 of 2356 tweet ids queried
852 of 2356 tweet ids queried
853 of 2356 tweet ids queried
854 of 2356 tweet ids queried
855 of 2356 tweet ids queried
856 of 2356 tweet ids queried
857 of 2356 tweet ids queried
858 of 2356 tweet ids queried
859 of 2356 tweet ids queried
860 of 2356 tweet ids queried
861 of 2356 tweet ids queried
862 of 2356 tweet ids queried
863 of 2356 tweet ids queried
864 of 2356 tweet ids queried
865 of 235

Rate limit reached. Sleeping for: 56


...

1699 of 2356 tweet ids queried
1700 of 2356 tweet ids queried
1701 of 2356 tweet ids queried
1702 of 2356 tweet ids queried
1703 of 2356 tweet ids queried
1704 of 2356 tweet ids queried
1705 of 2356 tweet ids queried
1706 of 2356 tweet ids queried
1707 of 2356 tweet ids queried
1708 of 2356 tweet ids queried
1709 of 2356 tweet ids queried
1710 of 2356 tweet ids queried
1711 of 2356 tweet ids queried
1712 of 2356 tweet ids queried
1713 of 2356 tweet ids queried
1714 of 2356 tweet ids queried
1715 of 2356 tweet ids queried
1716 of 2356 tweet ids queried
1717 of 2356 tweet ids queried
1718 of 2356 tweet ids queried
1719 of 2356 tweet ids queried
1720 of 2356 tweet ids queried
1721 of 2356 tweet ids queried
1722 of 2356 tweet ids queried
1723 of 2356 tweet ids queried
1724 of 2356 tweet ids queried
1725 of 2356 tweet ids queried
1726 of 2356 tweet ids queried
1726 was not found
1728 of 2356 tweet ids queried
1729 of 2356 tweet ids queried
1730 of 2356 tweet ids queried
1731 of 2356 tweet i

Rate limit reached. Sleeping for: 46


1801 of 2356 tweet ids queried
1802 of 2356 tweet ids queried
1803 of 2356 tweet ids queried
1804 of 2356 tweet ids queried
1805 of 2356 tweet ids queried
1806 of 2356 tweet ids queried
1807 of 2356 tweet ids queried
1808 of 2356 tweet ids queried
1809 of 2356 tweet ids queried
1810 of 2356 tweet ids queried
1811 of 2356 tweet ids queried
1812 of 2356 tweet ids queried
1813 of 2356 tweet ids queried
1814 of 2356 tweet ids queried
1815 of 2356 tweet ids queried
1816 of 2356 tweet ids queried
1817 of 2356 tweet ids queried
1818 of 2356 tweet ids queried
1819 of 2356 tweet ids queried
1820 of 2356 tweet ids queried
1821 of 2356 tweet ids queried
1822 of 2356 tweet ids queried
1823 of 2356 tweet ids queried
1824 of 2356 tweet ids queried
1825 of 2356 tweet ids queried
1826 of 2356 tweet ids queried
1827 of 2356 tweet ids queried
1828 of 2356 tweet ids queried
1829 of 2356 tweet ids queried
1830 of 2356 tweet ids queried
1831 of 2356 tweet ids queried
1832 of 2356 tweet ids queried
1833 of 

2066 of 2356 tweet ids queried
2067 of 2356 tweet ids queried
2068 of 2356 tweet ids queried
2069 of 2356 tweet ids queried
2070 of 2356 tweet ids queried
2071 of 2356 tweet ids queried
2072 of 2356 tweet ids queried
2073 of 2356 tweet ids queried
2074 of 2356 tweet ids queried
2075 of 2356 tweet ids queried
2076 of 2356 tweet ids queried
2077 of 2356 tweet ids queried
2078 of 2356 tweet ids queried
2079 of 2356 tweet ids queried
2080 of 2356 tweet ids queried
2081 of 2356 tweet ids queried
2082 of 2356 tweet ids queried
2083 of 2356 tweet ids queried
2084 of 2356 tweet ids queried
2085 of 2356 tweet ids queried
2086 of 2356 tweet ids queried
2087 of 2356 tweet ids queried
2088 of 2356 tweet ids queried
2089 of 2356 tweet ids queried
2090 of 2356 tweet ids queried
2091 of 2356 tweet ids queried
2092 of 2356 tweet ids queried
2093 of 2356 tweet ids queried
2094 of 2356 tweet ids queried
2095 of 2356 tweet ids queried
2096 of 2356 tweet ids queried
2097 of 2356 tweet ids queried
2098 of 

2331 of 2356 tweet ids queried
2332 of 2356 tweet ids queried
2333 of 2356 tweet ids queried
2334 of 2356 tweet ids queried
2335 of 2356 tweet ids queried
2336 of 2356 tweet ids queried
2337 of 2356 tweet ids queried
2338 of 2356 tweet ids queried
2339 of 2356 tweet ids queried
2340 of 2356 tweet ids queried
2341 of 2356 tweet ids queried
2342 of 2356 tweet ids queried
2343 of 2356 tweet ids queried
2344 of 2356 tweet ids queried
2345 of 2356 tweet ids queried
2346 of 2356 tweet ids queried
2347 of 2356 tweet ids queried
2348 of 2356 tweet ids queried
2349 of 2356 tweet ids queried
2350 of 2356 tweet ids queried
2351 of 2356 tweet ids queried
2352 of 2356 tweet ids queried
2353 of 2356 tweet ids queried
2354 of 2356 tweet ids queried
2355 of 2356 tweet ids queried
2356 of 2356 tweet ids queried
Total time elapsed:-2331.6993656158447


In [8]:
df_retweet = pd.read_csv('tweet_json.csv',sep=' ')
df_retweet.head()

,id,retweet_count,favorite_count
0,892420643555336193,8213,37692
1,892177421306343426,6075,32377
2,891815181378084864,4017,24383
3,891689557279858688,8370,41011
4,891327558926688256,9075,39211


In [9]:
twitter_arch

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [10]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Data Wrangling

#### Data Quality Issues
1. *Lot of missing values in in_reply_to status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns.*
2. *Extra '+0000' in timestamp column.*
3. *Trim the 'source' column to make it more readable.*
4. *expanded_urls won't help in analysis as it does not contain any important information (except for tweet ids which are already in place)*
5. *Some errors while fetching 'rating_numerator' column values from 'text' column.*
6. *'rating_denominator' must have all values equal to 10.*
7. *Weird names in 'names' column.*
8. *Change the datat

#### Define
1. *Dropping columns in_reply_to status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp.*

In [3]:
twitter_arch_copy = twitter_arch.copy()
twitter_arch_copy.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Code

In [4]:
#Dropping columns
twitter_arch_copy.drop(columns=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],inplace=True)

#### Test

In [5]:
twitter_arch_copy.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Define
2. Trim '+0000' from timestamp column values

#### Code

In [6]:
twitter_arch_copy['timestamp'] = twitter_arch_copy['timestamp'].str[:-6]

#### Test

In [7]:
twitter_arch_copy.timestamp

0       2017-08-01 16:23:56
1       2017-08-01 00:17:27
2       2017-07-31 00:18:03
3       2017-07-30 15:58:51
4       2017-07-29 16:00:24
5       2017-07-29 00:08:17
6       2017-07-28 16:27:12
7       2017-07-28 00:22:40
8       2017-07-27 16:25:51
9       2017-07-26 15:59:51
10      2017-07-26 00:31:25
11      2017-07-25 16:11:53
12      2017-07-25 01:55:32
13      2017-07-25 00:10:02
14      2017-07-24 17:02:04
15      2017-07-24 00:19:32
16      2017-07-23 00:22:39
17      2017-07-22 16:56:37
18      2017-07-22 00:23:06
19      2017-07-21 01:02:36
20      2017-07-20 16:49:33
21      2017-07-19 16:06:48
22      2017-07-19 03:39:09
23      2017-07-19 00:47:34
24      2017-07-18 16:08:03
25      2017-07-18 00:07:08
26      2017-07-17 16:17:36
27      2017-07-16 23:58:41
28      2017-07-16 20:14:00
29      2017-07-15 23:25:31
               ...         
2326    2015-11-17 00:24:19
2327    2015-11-17 00:06:54
2328    2015-11-16 23:23:41
2329    2015-11-16 21:54:18
2330    2015-11-16 2

#### Define
3. Make 'source' column more readable

#### Code

In [8]:
for src in range(len(twitter_arch_copy.source)):
    twitter_arch_copy['source'][src] = twitter_arch_copy['source'][src].split('>')[1][:-3]

H:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Test

In [9]:
twitter_arch_copy.source.value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

In [10]:
twitter_arch_copy.source.values

array(['Twitter for iPhone', 'Twitter for iPhone', 'Twitter for iPhone',
       ..., 'Twitter for iPhone', 'Twitter for iPhone',
       'Twitter for iPhone'], dtype=object)

#### Define
5. Dropping expanded_urls column

#### Code

In [11]:
twitter_arch_copy.drop(columns=['expanded_urls'],inplace=True)

#### Test

In [12]:
twitter_arch_copy.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

#### Define
5. Fix rating_numerator column values 

#### Code

In [13]:
import re
rating_num_idx = []
rating_num_idx_2 = []
for idx in range(len(twitter_arch_copy)):
    txt=twitter_arch_copy.iloc[idx].text
    text = re.findall(r'[0-9]+/10+',txt)
    if(len(text) == 1):
        value = text[0].split('/')[0]
        twitter_arch_copy.iloc[idx].rating_numerator = int(value)
    elif(len(text) < 1):
        rating_num_idx.append(idx)
    else:
        rating_num_idx_2.append(idx)

H:\anaconda\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
#rating_num_idx contains indixes of the format a/b and rating_num_idx_2 contains all indices where text contains more than one value of the format x/10.
len(rating_num_idx),len(rating_num_idx_2)

(16, 26)

In [15]:
for i in rating_num_idx:
    print(i,twitter_arch_copy.iloc[i].text)

342 @docmisterio account started on 11/15/15
433 The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd
516 Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx
902 Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
1120 Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
1228 Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1
1254 Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12
1274 From left to right:
Cletus, Jerome, Alejandro, Burp, &amp; Titson
None know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK
1351 Here is a whole flock of puppers.  60/50 I'll take the lot https://t.co/9dpcw6MdWa
1433 Happy Wednesd

In [16]:
#There are 16 entries where rating is given incorrectly i.e. of not the format set by WeRateDogs, hence dropping these rows.
twitter_arch_copy.drop(rating_num_idx,inplace=True)

In [17]:
#New shape of the twitter_arch_copy dataframe
twitter_arch_copy.shape

(2340, 11)

In [18]:
#Since rows were dropped in previous cell, indices are reset; hence gathering all indices where more than one rating is given.
import re
rating_num_idx_2 = []
for idx in range(len(twitter_arch_copy)):
    txt=twitter_arch_copy.iloc[idx].text
    text = re.findall(r'[0-9]+/10+',txt)
    if(len(text) > 1):
        rating_num_idx_2.append(idx)

In [19]:
len(rating_num_idx_2)

26

In [20]:
for i in rating_num_idx_2:
    print(i,twitter_arch_copy.iloc[i].text)

55 @roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s
558 RT @dog_rates: "Yep... just as I suspected. You're not flossing." 12/10 and 11/10 for the pup not flossing https://t.co/SuXcI9B7pQ
763 "Yep... just as I suspected. You're not flossing." 12/10 and 11/10 for the pup not flossing https://t.co/SuXcI9B7pQ
857 RT @dog_rates: Meet Eve. She's a raging alcoholic 8/10 (would b 11/10 but pupper alcoholism is a tragic issue that I can't condone) https:/…
1003 This is Bookstore and Seaweed. Bookstore is tired and Seaweed is an asshole. 10/10 and 7/10 respectively https://t.co/eUGjGjjFVJ
1217 Meet Travis and Flurp. Travis is pretty chill but Flurp can't lie down properly. 10/10 &amp; 8/10
get it together Flurp https://t.co/Akzl5ynMmE
1350 This is Socks. That water pup w the super legs just splashed him. Socks did not appreciate that. 9/10 and 2/10 https://t.co/8rc5I22bBf
1449 This may be the greatest video I've ever been sent. 4/10 for Charles the pupp

In [21]:
#From the text above, storing indices in second_rating where second rating is the correct rating given
second_rating = [55,558,763,1498,1885,2256]

In [22]:
for i in rating_num_idx_2:
    if i in second_rating:
        txt = twitter_arch_copy.iloc[i].text
        text = re.findall(r'[0-9]+/10+',txt)
        text = text[1].split('/')[0]
        twitter_arch_copy.iloc[i].rating_numerator = int(text)
    else:
        txt = twitter_arch_copy.iloc[i].text
        text = re.findall(r'[0-9]+/10+',txt)
        text = text[0].split('/')[0]
        twitter_arch_copy.iloc[i].rating_numerator = int(text)

H:\anaconda\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [24]:
#Now the dataframe contains some really huge values, replacing them with NaN first and then with the average numerator value.
twitter_arch_copy.rating_numerator.replace([75,420,50,26,182,27,960,1776,666],np.NaN,inplace=True)

In [29]:
numerator_mean = int(twitter_arch_copy.rating_numerator.mean())

In [31]:
twitter_arch_copy.rating_numerator.fillna(numerator_mean,inplace=True)

In [36]:
twitter_arch_copy.rating_numerator.isna().sum()

0

In [44]:
#Convert rating_numerator to integer values
twitter_arch_copy.rating_numerator = twitter_arch_copy.rating_numerator.astype(int)

**Test**

In [45]:
twitter_arch_copy.rating_numerator.value_counts()

12    558
10    472
11    463
13    351
9     158
8     102
7      55
14     54
5      37
6      32
3      19
4      16
1       9
2       9
0       2
15      2
17      1
Name: rating_numerator, dtype: int64

**Define**
6. Set all rating_denominator values to 10

**Code**

In [47]:
twitter_arch_copy.rating_denominator = 10

**Test**

In [48]:
twitter_arch_copy.rating_denominator.value_counts()

10    2340
Name: rating_denominator, dtype: int64

**Define**
7. Fix name column values

**Code**

In [83]:
twitter_arch_copy.name.value_counts()

None          732
a              54
Charlie        12
Oliver         11
Lucy           11
Cooper         11
Penny          10
Tucker         10
Lola           10
Winston         9
Bo              9
Sadie           8
the             8
Buddy           7
an              7
Bailey          7
Daisy           7
Toby            7
Dave            6
Stanley         6
Leo             6
Oscar           6
Bella           6
Jax             6
Rusty           6
Milo            6
Scout           6
Jack            6
Koda            6
Larry           5
             ... 
Thor            1
Snoopy          1
Tater           1
Nimbus          1
Gert            1
Reptar          1
Terrenth        1
Nigel           1
Tassy           1
Sweets          1
Longfellow      1
Travis          1
Obie            1
Theo            1
Dylan           1
Dug             1
Millie          1
Carter          1
Dex             1
Daniel          1
Darby           1
Stephanus       1
Maks            1
Marty           1
Jett      

In [84]:
twitter_arch_copy.name.replace(['a','an'],None,inplace=True)

**Test**

In [85]:
twitter_arch_copy.name.value_counts()

None          757
Oliver         13
Charlie        12
the            12
Lucy           11
Cooper         11
Tucker         10
Lola           10
Penny          10
Bo              9
Winston         9
Daisy           8
Sadie           8
Bailey          7
Buddy           7
Toby            7
Stanley         6
Oscar           6
Leo             6
Milo            6
Scout           6
Jax             6
Dave            6
Bella           6
Koda            6
Jack            6
Rusty           6
Oakley          5
George          5
Phil            5
             ... 
Gert            1
Kirk            1
Humphrey        1
Boston          1
Rilo            1
Clyde           1
Shooter         1
Jo              1
Maks            1
Theo            1
Dylan           1
Dug             1
Millie          1
Carter          1
Tassy           1
Dex             1
Darby           1
Stephanus       1
Marty           1
Suki            1
Jett            1
Brady           1
Kaiya           1
Vince           1
Vinscent  

In [86]:
twitter_arch_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2340 non-null int64
timestamp             2340 non-null object
source                2340 non-null object
text                  2340 non-null object
rating_numerator      2340 non-null int32
rating_denominator    2340 non-null int64
name                  2340 non-null object
doggo                 2340 non-null object
floofer               2340 non-null object
pupper                2340 non-null object
puppo                 2340 non-null object
dtypes: int32(1), int64(2), object(8)
memory usage: 137.1+ KB


In [89]:
twitter_arch_copy.timestamp = pd.to_datetime(twitter_arch_copy.timestamp)

In [91]:
twitter_arch_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2340 non-null int64
timestamp             2340 non-null datetime64[ns]
source                2340 non-null object
text                  2340 non-null object
rating_numerator      2340 non-null int32
rating_denominator    2340 non-null int64
name                  2340 non-null object
doggo                 2340 non-null object
floofer               2340 non-null object
pupper                2340 non-null object
puppo                 2340 non-null object
dtypes: datetime64[ns](1), int32(1), int64(2), object(7)
memory usage: 146.2+ KB
